## Sproj Board Scheduler

An attempt to write a script that would read faculty and students availability for the boards week, and then schedule all boards, accounting for their target composition, and student availability.

Unfortunately some of the data used by this script (names) is sensitive, and had to be stored outside of git.

In [1]:
import pandas as pd
import time
import copy
import random
from IPython.display import HTML, display

In [2]:
hideNames = 1 # set 0 for troubleshooting, set 1 before githubbing

In [3]:
composition = pd.read_csv('../../sensitive_data/boards.csv')
students = pd.read_csv('../../sensitive_data/students.csv')
facultin = pd.read_csv('../../sensitive_data/faculty.csv')

In [4]:
# Create time grid
days = ['m','t','w','r','f']
dayName = {'m':'Mon','t':'Tue','w':'Wed','r':'Thu','f':'Fri'}

startTime = 9
endTime = 18
grid = [] # Array of tuples: day (as a letter), hour (as a 24h int number)
for d in days:
    for t in range(startTime,endTime):
        grid = grid + [(d,t)]
print(grid[:3])

facultin.time = facultin.time.apply(lambda x: max(x,startTime))
#print(facultin)
# Note that the script doesn't check times outside of start-end window, so all ins and outs should happen either
# within this window, or before the day starts (to add or remove an entire day)

[('m', 9), ('m', 10), ('m', 11)]


In [5]:
class Faculty:
    def __init__(self,name):
        self.name = name
        self.avail = []
        
    def __str__(self):
        return "%12s \t" % (self.name) + ''.join(['%d' % i for i in self.avail])
    
    def initAvail(self,grid,facultin):
        self.avail = [1]*len(grid)
        status = 1
        for ig in range(len(grid)):
            lin = facultin.loc[(facultin['name']==self.name) & 
                               (facultin['type']=='in') & 
                               (facultin['day']==grid[ig][0]) &
                               (facultin['time']==grid[ig][1])]
            if lin.shape[0]>0:
                status = 1
            lout = facultin.loc[(facultin['name']==self.name) & 
                               (facultin['type']=='out') & 
                               (facultin['day']==grid[ig][0]) &
                               (facultin['time']==grid[ig][1])]
            if lout.shape[0]>0:
                status = 0
            self.avail[ig] = status
            
    def updateAvail(self,grid,g,newVal=0):
        self.avail = [self.avail[i] if grid[i]!=g else newVal for i in range(len(self.avail))]
        
    def book(self,ig):
        self.avail[ig] = 0
        
        
faculty = []
for fn in composition['advisor'].append(composition['mem2']).append(composition['mem3']).unique():
    f = Faculty(fn)
    f.initAvail(grid,facultin)
    for d in days:
        f.updateAvail(grid,(d,12)) # Book lunch breaks
        if d=='w': # Wed is an advising day, so take it out completely
            for ih in range(1,25):
                f.updateAvail(grid,(d,ih),0)
    faculty.append(f)    

for f in faculty:
    print(f)

        Jude 	111011000111011000000000000000011111111011111
    Tibbetts 	000000000111011111000000000000000000000000000
   Robertson 	111011111111011111000000000111011111111011111
     Collins 	111011111111011111000000000111011111111011111
     Keesing 	111011111111011111000000000000000000000000000
   Khakhalin 	111011111111011111000000000111011111111011111
      Perron 	111011111111011111000000000111011111111011111
     Bennett 	111011111111011111000000000111011111111011111
      Dueker 	111011111111001100000000000111011111100000111
        Jain 	000000000000000100000000000000000100000000000
      Scalzo 	000000000000001110000000000000000100011010000


In [6]:
class Board:
    '''Board object'''
    
    def __init__(self,stuff): # Creator
        self.student = stuff['first'] + ' ' + stuff['last']
        if hideNames:
            self.student = ''.join(random.sample(self.student.lower(),len(self.student)))
        self.members = list(stuff[['advisor','mem2','mem3']])
        self.email = stuff['email']
        self.type = stuff['type']
        self.avail = []  # Placeholder: availability grid
        self.time = []
        
    def __str__(self):
        s = "%20s" % (self.student) + '\t' 
        s += ' '.join([m[:4] for m in self.members]) + '\t' # Shortened version
        s += ''.join(['%d' % i for i in self.avail])
        return s
    
    def initTimes(self,grid,students):
        '''Sets boards initial options, based on students availability'''
        data = students.loc[students.email==self.email]
        self.avail = [1]*len(grid)
        for i in range(data.shape[0]):
            h = data.iloc[i]
            if h['class'][:3]!='BIO':
                #print('Included:',[_ for _ in data.iloc[i]])
                for d in days:
                    if not pd.isna(h[d]): # Class meets on this day
                        #print(h['class'])
                        times = h['time'].split('-') # String of two times
                        for it in range(2):
                            if times[it][0]==' ':
                                times[it] = times[it][1:] # Remove leading spaces
                        tstart = time.strptime(times[0], "%I:%M %p")
                        tend   = time.strptime(times[1], "%I:%M %p")
                        for ig in range(len(grid)):
                            g = grid[ig]
                            if g[0]==d: # If this day
                                if g[1]>=tstart.tm_hour and g[1]<=tend.tm_hour: # If classes during this hour
                                    self.avail[ig] = 0
                                
    def refreshFac(self,faculty):
        '''Filters boards based on current faculty availability'''
        for f in faculty:
            for facname in self.members:
                if f.name==facname:
                    self.avail = [self.avail[i]*f.avail[i] for i in range(len(self.avail))]
          
    
boards = []
print('Student availability:')
for ib in range(len(composition)):
    b = Board(composition.loc[ib,])
    b.initTimes(grid,students)
    print(b)
    boards.append(b)
    
for b in boards:
    b.refreshFac(faculty)
    
# Rearrange from those that are harder to schedule to those that are easier
niceness = [sum(b.avail) for b in boards]
ind = [i for _,i in sorted(zip(niceness,range(len(niceness))))]
boards = [boards[i] for i in ind]

print('\nFull Board availability:')
for b in boards:
    print(b)

Student availability:
           eadioyn s	Jude Coll Kees	111111111111111111111100000111111111100111111
    azqeaiugelbvsr a	Tibb Jude Khak	111111111111100111111111111111100111111111111
         a afdnuzrie	Robe Jude Kees	111111111111111100111111111111111111000000011
         ibmrslaoawl	Coll Robe Kees	110001111100111111110001111100001111111111111
         paoeet korc	Kees Coll Robe	111111111111111100111111111111111111000000011
         iheifdcda n	Khak Jude Benn	111111111100111100111111111100111100111111111
         ynamaco nev	Perr Benn Khak	111111111100000111111111111111100111111111111
          rbayen ntb	Coll Kees Perr	100000011110001111111111111110001111111111111
       b uircouncdni	Benn Perr Khak	111111111111100111111111111111100111111111111
       acyh cmllecit	Kees Robe Coll	111100001100011111111100001111111111111111111
    ysraot innltpoel	Duek Kees Perr	111100011111111100111111111111111111000000011
    mliehottszbaa he	Kees Coll Khak	1111111001001111111111000011001111111111

In [7]:
# Backup, just so that the cell below could be rerun without ruining the data
bb = copy.deepcopy(boards)
bf = copy.deepcopy(faculty)

In [8]:
boards = copy.deepcopy(bb)
faculty = copy.deepcopy(bf)

for b in boards:
    b.refreshFac(faculty)
    temp = [i for i in range(len(b.avail)) if (b.avail[i]==1)]
    if len(temp)==0:
        print('Cannot solve the puzzle for this board:')
        print(b)
        break
    ig = min(temp)
    b.time = grid[ig]
    b.avail[ig] = 0
    for f in faculty:
        if f.name in b.members:
            f.book(ig)
    print('%s\t%s\t' % (dayName[grid[ig][0]],grid[ig][1]), end='')
    print(b)
    
#for b in boards:
#    print(b)

Tue	15	      aehinc hrhakrn	Perr Duek Jain	000000000000000000000000000000000100000000000
Thu	15	    aeeeaw npnsetjue	Perr Benn Jain	000000000000000000000000000000000000000000000
Tue	9	    azqeaiugelbvsr a	Tibb Jude Khak	000000000011000000000000000000000000000000000
Tue	11	        e lgeyctnnde	Jude Tibb Coll	000000000000011000000000000000000000000000000
Tue	10	        ibn auolijsg	Jude Kees Tibb	000000000000011000000000000000000000000000000
Tue	14	 aaaltalsacmmane ert	Perr Jude Scal	000000000000000000000000000000000000011010000
Tue	13	        amec iehluhl	Jude Tibb Khak	000000000000000000000000000000000000000000000
Mon	9	       ylecnegiraam 	Kees Jude Khak	011011000000000000000000000000000000000000000
Mon	13	          rksmano ee	Kees Jude Khak	000001000000000000000000000000000000000000000
Mon	16	          rbayen ntb	Coll Kees Perr	000000001100000011000000000000000000000000000
Mon	10	           eadioyn s	Jude Coll Kees	001001000000000000000000000000000000000000000
Mon	11	         a afdnu

In [9]:
ind = [min([i for i in range(len(grid)) if grid[i]==b.time]) for b in boards]
ind = [j for (i,j) in sorted(zip(ind,range(len(ind))))]
boards = [boards[i] for i in ind]

def printboard(b,mode="none"):
    if mode=="none":
        print("%22s\t%12s\t%s\t%d\t" % (b.student,b.type,dayName[b.time[0]],b.time[1]),end='')
        for fn in b.members:
            print("%10s\t" % (fn),end='')
        print()
    elif mode=="tabs":
        print("%s\t%s\t%s\t%d\t" % (b.student,b.type,dayName[b.time[0]],b.time[1]),end='')
        for fn in b.members:
            print("%s\t" % (fn),end='')
        print()
    elif mode=="html":
        s = ''
        s += "<tr><td>%s</td><td>%s</td><td>%s %d</td><td>" % (b.student,b.type,dayName[b.time[0]],b.time[1])
        for fn in b.members:
            s += "%s " % (fn)
        s += "</td></tr>"
        display(HTML(s))

for b in boards:
    printboard(b,mode="tabs")

ylecnegiraam 	moderation	Mon	9	Keesing	Jude	Khakhalin	
aybhmrnba agt	midway	Mon	9	Robertson	Perron	Dueker	
eadioyn s	final	Mon	10	Jude	Collins	Keesing	
donazmya dnr	midway	Mon	10	Robertson	Perron	Dueker	
a afdnuzrie	final	Mon	11	Robertson	Jude	Keesing	
ynamaco nev	midway	Mon	11	Perron	Bennett	Khakhalin	
rksmano ee	moderation	Mon	13	Keesing	Jude	Khakhalin	
 rashanbiczictirn	midway	Mon	13	Dueker	Collins	Robertson	
ezip-epntliolerohaf k	moderation	Mon	14	Keesing	Jude	Khakhalin	
 ibmrslaoawl	final	Mon	15	Collins	Robertson	Keesing	
b uircouncdni	midway	Mon	15	Bennett	Perron	Khakhalin	
rbayen ntb	midway	Mon	16	Collins	Keesing	Perron	
acyh cmllecit	midway	Mon	17	Keesing	Robertson	Collins	
azqeaiugelbvsr a	final	Tue	9	Tibbetts	Jude	Khakhalin	
ysraot innltpoel	midway	Tue	9	Dueker	Keesing	Perron	
ibn auolijsg	midway	Tue	10	Jude	Keesing	Tibbetts	
e lgeyctnnde	midway	Tue	11	Jude	Tibbetts	Collins	
amec iehluhl	midway	Tue	13	Jude	Tibbetts	Khakhalin	
l iudanelsars	midway	Tue	13	Keesing	Collins	Perron

In [10]:
#Sort by faculty:

for f in faculty:
    print()
    print(f.name)
    for b in boards:
        if f.name in b.members:
            printboard(b,mode="tabs")


Jude
ylecnegiraam 	moderation	Mon	9	Keesing	Jude	Khakhalin	
eadioyn s	final	Mon	10	Jude	Collins	Keesing	
a afdnuzrie	final	Mon	11	Robertson	Jude	Keesing	
rksmano ee	moderation	Mon	13	Keesing	Jude	Khakhalin	
ezip-epntliolerohaf k	moderation	Mon	14	Keesing	Jude	Khakhalin	
azqeaiugelbvsr a	final	Tue	9	Tibbetts	Jude	Khakhalin	
ibn auolijsg	midway	Tue	10	Jude	Keesing	Tibbetts	
e lgeyctnnde	midway	Tue	11	Jude	Tibbetts	Collins	
amec iehluhl	midway	Tue	13	Jude	Tibbetts	Khakhalin	
aaaltalsacmmane ert	midway	Tue	14	Perron	Jude	Scalzo	
iheifdcda n	midway	Thu	13	Khakhalin	Jude	Bennett	
aqisranat 	midway	Thu	14	Khakhalin	Jude	Bennett	
hemie ljtsw	midway	Thu	16	Collins	Jude	Bennett	
hiomcder arleac	midway	Thu	17	Khakhalin	Jude	Bennett	

Tibbetts
azqeaiugelbvsr a	final	Tue	9	Tibbetts	Jude	Khakhalin	
ibn auolijsg	midway	Tue	10	Jude	Keesing	Tibbetts	
e lgeyctnnde	midway	Tue	11	Jude	Tibbetts	Collins	
amec iehluhl	midway	Tue	13	Jude	Tibbetts	Khakhalin	

Robertson
aybhmrnba agt	midway	Mon	9	Robertson	Per